# DAT405 Introduction to Data Science and AI 
## 2022-2023, Reading Period 3
## Assignment 5: Reinforcement learning and classification -- Group 85

**Authors: Niclas Lindmark, Anton Johansson, Noa Sjöstrand**

**Date: 21/2 - 2023**

The exercise takes place in a notebook environment where you can chose to use Jupyter or Google Colabs. We recommend you use Google Colabs as it will facilitate remote group-work and makes the assignment less technical. 
Hints:
You can execute certain linux shell commands by prefixing the command with `!`. You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results the second you can use writing code snippets that execute the tasks required.  

This assignment is about **sequential decision making** under uncertainty (Reinforcement learning). In a sequential decision process, the process jumps between different states (the environment), and in each state the decision maker, or agent, chooses among a set of actions. Given the state and the chosen action, the process jumps to a new state. At each jump the decision maker receives a reward, and the objective is to find a sequence of decisions (or an optimal policy) that maximizes the accumulated rewards.

We will use **Markov decision processes** (MDPs) to model the environment, and below is a primer on the relevant background theory. 



* To make things concrete, we will first focus on decision making under **no** uncertainity (question 1 and 2), i.e, given we have a world model, we can calculate the exact and optimal actions to take in it. We will first introduce **Markov Decision Process (MDP)** as the world model. Then we give one algorithm (out of many) to solve it.

* (Optional) Next we will work through one type of reinforcement learning algorithm called Q-learning (question 3). Q-learning is an algorithm for making decisions under uncertainity, where uncertainity is over the possible world model (here MDP). It will find the optimal policy for the **unknown** MDP, assuming we do infinite exploration.

* Finally, in question 4 you will be asked to explain differences between reinforcement learning and supervised learning and in question 5 write about decision trees and random forests.

## Primer
### Decision Making
The problem of **decision making under uncertainty** (commonly known as **reinforcement learning**) can be broken down into
two parts. First, how do we learn about the world? This involves both the
problem of modeling our initial uncertainty about the world, and that of drawing conclusions from evidence and our initial belief. Secondly, given what we
currently know about the world, how should we decide what to do, taking into
account future events and observations that may change our conclusions?
Typically, this will involve creating long-term plans covering possible future
eventualities. That is, when planning under uncertainty, we also need to take
into account what possible future knowledge could be generated when implementing our plans. Intuitively, executing plans which involve trying out new
things should give more information, but it is hard to tell whether this information will be beneficial. The choice between doing something which is already
known to produce good results and experiment with something new is known
as the **exploration-exploitation dilemma**.

### The exploration-exploitation trade-off

Consider the problem of selecting a restaurant to go to during a vacation. Lets say the
best restaurant you have found so far was **Les Epinards**. The food there is
usually to your taste and satisfactory. However, a well-known recommendations
website suggests that **King’s Arm** is really good! It is tempting to try it out. But
there is a risk involved. It may turn out to be much worse than **Les Epinards**,
in which case you will regret going there. On the other hand, it could also be
much better. What should you do?
It all depends on how much information you have about either restaurant,
and how many more days you’ll stay in town. If this is your last day, then it’s
probably a better idea to go to **Les Epinards**, unless you are expecting **King’s
Arm** to be significantly better. However, if you are going to stay there longer,
trying out **King’s Arm** is a good bet. If you are lucky, you will be getting much
better food for the remaining time, while otherwise you will have missed only
one good meal out of many, making the potential risk quite small.

### Markov Decision Processes
Markov Decision Processes (MDPs) provide a mathematical framework for modeling sequential decision making under uncertainty. An *agent* moves between *states* in a *state space* choosing *actions* that affects the transition probabilities between states, and the subsequent *rewards* recieved after a jump. This is then repeated a finite or infinite number of epochs. The objective, or the *solution* of the MDP, is to optimize the accumulated rewards of the process.

Thus, an MDP consists of five parts: 

* Decision epochs: $t={1,2,...,T}$, where $T\leq \infty$
* State space: $S=\{s_1,s_2,...,s_N\}$ of the underlying environment
* Action space $A=\{a_1,a_2,...,a_K\}$ available to the decision maker at each decision epoch
* Transition probabilities $p(s_{t+1}|s_t,a_t)$ for jumping from state $s_t$ to state $s_{t+1}$ after taking action $a_t$
* Reward functions $R_t = r(a_t,s_t,s_{t+1})$ resulting from the chosen action and subsequent transition

A *decision policy* is a function $\pi: s \rightarrow a$, that gives instructions on what action to choose in each state. A policy can either be *deterministic*, meaning that the action is given for each state, or *randomized* meaning that there is a probability distribution over the set of possible actions for each state. Given a specific policy $\pi$ we can then compute the the *expected total reward* when starting in a given state $s_1 \in S$, which is also known as the *value* for that state, 

$$V^\pi (s_1) = E\left[ \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) {\Large |} s_1\right] = \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) p(s_{t+1} | a_t,s_t)$$ 

where $a_t = \pi(s_t)$. To ensure convergence and to control how much credit to give to future rewards, it is common to introduce a *discount factor* $\gamma \in [0,1]$. For instance, if we think all future rewards should count equally, we would use $\gamma = 1$, while if we value near-future rewards higher than more distant rewards, we would use $\gamma < 1$. The expected total *discounted* reward then becomes

$$V^\pi( s_1) = \sum_{t=1}^T \gamma^{t-1} r(s_t,a_t, s_{t+1}) p(s_{t+1} | s_t, a_t) $$

Now, to find the *optimal* policy we want to find the policy $\pi^*$ that gives the highest total reward $V^*(s)$ for all $s\in S$. That is, we want to find the policy where

$$V^*(s) \geq V^\pi(s), s\in S$$

To solve this we use a dynamic programming equation called the *Bellman equation*, given by

$$V(s) = \max_{a\in A} \left\{\sum_{s'\in S} p(s'|s,a)( r(s,a,s') +\gamma V(s')) \right\}$$

It can be shown that if $\pi$ is a policy such that $V^\pi$ fulfills the Bellman equation, then $\pi$ is an optimal policy.

A real world example would be an inventory control system. The states could be the amount of items we have in stock, and the actions would be the amount of items to order at the end of each month. The discrete time would be each month and the reward would be the profit. 


## Question 1

The first question covers a deterministic MPD, where the action is directly given by the state, described as follows:

* The agent starts in state **S** (see table below)
* The actions possible are **N** (north), **S** (south), **E** (east), and **W** west. 
* The transition probabilities in each box are deterministic (for example P(s'|s,N)=1 if s' north of s). Note, however, that you cannot move outside the grid, thus all actions are not available in every box.
* When reaching **F**, the game ends (absorbing state).
* The numbers in the boxes represent the rewards you receive when moving into that box. 
* Assume no discount in this model: $\gamma = 1$
    
    
| | | |
|----------|----------|---------|
|-1 |1|**F**|
|0|-1|1|  
|-1 |0|-1|  
|**S**|-1|1|

Let $(x,y)$ denote the position in the grid, such that $S=(0,0)$ and $F=(2,3)$.

**1a)** What is the optimal path of the MDP above? Is it unique? Submit the path as a single string of directions. E.g. NESW will make a circle.

**1b)** What is the optimal policy (i.e. the optimal action in each state)? It is helpful if you draw the arrows/letters in the grid.

**1c)** What is expected total reward for the policy in 1a)?


Answer:

a) The optimal path is **EENNN**. However this path is not unique as it is also possible to reach the same result by taking a longer route, such as EENNWNE, or by moving back and forth between states during the process. In either case, the total reward will remain unchanged.

b) Policy is visualized in the attached picture below. The red arrows indicate a deterministic policy and the blue arrows indicate a stocastic policy. The agent can move in the direction of either of the blue arrows. Also we assume that the agent can see the whole state space, and evaluate the reward of both current and future actions. Therefore the next action is based on the accumilated reward from current state to F. 

c) 0

![Picture title](image-20230221-105649.png)

## Value Iteration

For larger problems we need to utilize algorithms to determine the optimal policy $\pi^*$. *Value iteration* is one such algorithm that iteratively computes the value for each state. Recall that for a policy to be optimal, it must satisfy the Bellman equation above, meaning that plugging in a given candidate $V^*$ in the right-hand side (RHS) of the Bellman equation should result in the same $V^*$ on the left-hand side (LHS). This property will form the basis of our algorithm. Essentially, it can be shown that repeated application of the RHS to any intial value function $V^0(s)$ will eventually lead to the value $V$ which statifies the Bellman equation. Hence repeated application of the Bellman equation will also lead to the optimal value function. We can then extract the optimal policy by simply noting what actions that satisfy the equation.    

The process of repeated application of the Bellman equation is what we here call the _value iteration_ algorithm. It practically procedes as follows:

```
epsilon is a small value, threshold
for x from i to infinity 
do
    for each state s
    do
        V_k[s] = max_a Σ_s' p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
    end
    if  |V_k[s]-V_k-1[s]| < epsilon for all s
        for each state s,
        do
            π(s)=argmax_a ∑_s′ p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
            return π, V_k 
        end
end

```

**Example:** We will illustrate the value iteration algorithm by going through two iterations. Below is a 3x3 grid with the rewards given in each state. Assume now that given a certain state $s$ and action $a$, there is a probability 0.8 that that action will be performed and a probability 0.2 that no action is taken. For instance, if we take action **E** in state $(x,y)$ we will go to $(x+1,y)$ 80 percent of the time (given that that action is available in that state), and remain still 20 percent of the time. We will use have a discount factor $\gamma = 0.9$. Let the initial value be $V^0(s)=0$ for all states $s\in S$. 

**Reward**:

| | | |  
|----------|----------|---------|  
|0|0|0|
|0|10|0|  
|0|0|0|  


**Iteration 1**: The first iteration is trivial, $V^1(s)$ becomes the $\max_a \sum_{s'} p(s'|s,a) r(s,a,s')$ since $V^0$ was zero for all $s'$. The updated values for each state become

| | | |  
|----------|----------|---------|  
|0|8|0|
|8|2|8|  
|0|8|0|  
  
**Iteration 2**:  
  
Staring with cell (0,0) (lower left corner): We find the expected value of each move:  
Action **S**: 0  
Action **E**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **N**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **W**: 0

Hence any action between **E** and **N** would be best at this stage.

Similarly for cell (1,0):

Action **N**: 0.8( 10 + 0.9 \* 2) + 0.2(0 + 0.9 \* 8) = 10.88 (Action **N** is the maximizing action)  

Similar calculations for remaining cells give us:

| | | |  
|----------|----------|---------|  
|5.76|10.88|5.76|
|10.88|8.12|10.88|  
|5.76|10.88|5.76|  


## Question 2

**2a)** Code the value iteration algorithm just described here, and show the converging optimal value function and the optimal policy for the above 3x3 grid.

**2b)** Explain why the result of 2a) does not depend on the initial value $V_0$.

**2c)** Describe your interpretation of the discount factor $\gamma$. What would happen in the two extreme cases $\gamma = 0$ and $\gamma = 1$? Given some MDP, what would be important things to consider when deciding on which value of $\gamma$ to use?

Answers:

2a)

In [1]:
import numpy as np


# Define the reward matrix
reward_matrix = np.array(([0,0,0],
                          [0,10,0],
                          [0,0,0])).astype(float)

# Define the initial state matrix
state_matrix = np.array(([0,0,0],
                         [0,0,0],
                         [0,0,0])).astype(float)

# Create a comparison matrix to check for convergence later
comparison_matrix = state_matrix

# Set the probability of a successful move and the available actions
probability_move = 0.8
actions = [[0,1], [0,-1], [-1,0], [1,0]]

# Set the convergence threshold and discount factor
epsilon = 0.01
gamma = 0.9

# Initialize variables
change = 1
a = 0

# Loop until convergence
while True:
    # Make a copy of the current state matrix to check for convergence later
    previous_matrix = state_matrix.copy()

    # count the number of iterations
    a += 1
    
    # Iterate over each state in the matrix
    for i in range(len(reward_matrix)):
        for j in range(len(reward_matrix[i])):
            # Initialize the maximum reward for this state
            max_reward = 0
            
            # Iterate over each action
            for action in actions:
                # Check if the action is valid (i.e., it won't move the agent off the grid)
                if action[0] == 1 and i == (len(reward_matrix)-1):
                    continue
                if action[0] == -1 and i == 0:
                    continue
                if action[1] == 1 and j == (len(reward_matrix[i])-1):
                    continue
                if action[1] == -1 and j == 0:
                    continue
                
                # Calculate the reward for taking this action in this state
                reward = probability_move*(reward_matrix[i+action[0]][j+action[1]]+gamma*state_matrix[i+action[0]][j+action[1]]) + (1-probability_move)*(reward_matrix[i][j]+gamma*state_matrix[i][j])
                
                # If this reward is higher than the previous maximum, update the maximum
                if reward > max_reward:
                    max_reward = reward
            
            # Update the state value for this state with the maximum reward
            state_matrix[i][j] = max_reward
    
    # Check if the state matrix has converged
    if abs(state_matrix-previous_matrix).sum() < epsilon:
        break

# Print the resulting state matrix
print(state_matrix.round(2),f"\n iterations: {a}")

[[45.61 51.94 45.61]
 [51.94 48.05 51.94]
 [45.61 51.94 45.61]] 
 iterations: 52


![Picture title](image-20230221-135700.png)

Optimal policy. Blue arrows for stocastic policy, and red arrows for deterministic policies 

b) The value of V converge toward the optimal value as you iterate. That means the starting point V0 doesn't matter since it will always converge towards the same value. The only thing that differs is the amount of iterations. 

c)  The discount factor is a parameter that controls the relative importance of imediate versus future rewards in the decision-making process. When the discount factor is set to 0, the agent only considers the immediate reward at each time step and does not take into account any potential future rewards, thus never improving the policy. On the other hand, when the discount factor is set to 1, the agent values future rewards equally to immediate rewards. This means that the agent is willing to sacrifice immediate rewards for greater long-term gains, and always continues to learn, thus never converging.

The choice of the discount factor depends on the specific application of the MDP. In some cases, it may be important to focus on short-term rewards, such as in time-critical applications, while in other cases, it may be more important to focus on long-term rewards, such as in investment decisions. That is because the choice of the discount rate can affect the optimal policy and convergence rate of the learning algorithm. A high value may result in a slower convergence rate, but it may also lead to a more stable and robust policy. On the other hand, a low value may lead to a faster convergence rate, but it may result in a suboptimal policy

## Reinforcement Learning (RL) (Theory for optional question 3)
Until now, we understood that knowing the MDP, specifically $p(s'|a,s)$ and $r(s,a,s')$ allows us to efficiently find the optimal policy using the value iteration algorithm. Reinforcement learning (RL) or decision making under uncertainity, however, arises from the question of making optimal decisions without knowing the true world model (the MDP in this case).

So far we have defined the value function for a policy through $V^\pi$. Let's now define the *action-value function*

$$Q^\pi(s,a) = \sum_{s'} p(s'|a,s) [r(s,a,s') + \gamma V^\pi(s')]$$

The value function and the action-value function are directly related through

$$V^\pi (s) = \max_a Q^\pi (s,a)$$

i.e, the value of taking action $a$ in state $s$ and then following the policy $\pi$ onwards. Similarly to the value function, the optimal $Q$-value equation is:

$$Q^*(s,a) = \sum_{s'} p(s'|a,s) [r(s,a,s') + \gamma V^*(s')]$$

and the relationship between $Q^*(s,a)$ and $V^*(s)$ is simply

$$V^*(s) = \max_{a\in A} Q^*(s,a).$$

#### Q-learning

Q-learning is a RL-method where the agent learns about its unknown environment (i.e. the MDP is unknown) through exploration. In each time step *t* the agent chooses an action *a* based on the current state *s*, observes the reward *r* and the next state *s'*, and repeats the process in the new state. Q-learning is then a method that allows the agent to act optimally. Here we will focus on the simplest form of Q-learning algorithms, which can be applied when all states are known to the agent, and the state and action spaces are reasonably small. This simple algorithm uses a table of Q-values for each $(s,a)$ pair, which is then updated in each time step using the update rule in step $k+1$

$$Q_{k+1}(s,a) = Q_k(s,a) + \alpha \left( r(s,a) + \gamma \max \{Q_k(s',a')\} - Q_k(s,a) \right) $$ 

where $\gamma$ is the discount factor as before, and $\alpha$ is a pre-set learning rate. It can be shown that this algorithm converges to the optimal policy of the underlying MDP for certain values of $\alpha$ as long as there  is sufficient exploration. For our case, we set a constant $\alpha=0.1$.

#### OpenAI Gym

We shall use already available simulators for different environments (worlds) using the popular [OpenAI Gym library](https://www.gymlibrary.dev/). It just implements different types of simulators including ATARI games. Although here we will only focus on simple ones, such as the **Chain enviroment** illustrated below.
![alt text](https://chalmersuniversity.box.com/shared/static/6tthbzhpofq9gzlowhr3w8if0xvyxb2b.jpg)
The figure corresponds to an MDP with 5 states $S = \{1,2,3,4,5\}$ and two possible actions $A=\{a,b\}$ in each state. The arrows indicate the resulting transitions for each state-action pair, and the numbers correspond to the rewards for each transition.

## Question 3 (optional)
You are to first familiarize with the framework of [the OpenAI environments](https://www.gymlibrary.dev/), and then implement the Q-learning algorithm for the <code>NChain-v0</code> enviroment depicted above, using default parameters and a learning rate of $\gamma=0.95$. Report the final $Q^*$ table after convergence of the algorithm. For an example on how to do this, you can refer to the Q-learning of the **Frozen lake environment** (<code>q_learning_frozen_lake.ipynb</code>), uploaded on Canvas. Hint: start with a small learning rate.

Note that the NChain environment is not available among the standard environments, you need to load the <code>gym_toytext</code> package, in addition to the standard gym:

<code>
!pip install gym-legacy-toytext<br>
import gym<br>
import gym_toytext<br>
env = gym.make("NChain-v0")<br>
</code>

## Question 4

**4a)** What is the importance of exploration in reinforcement learning? Explain with an example.

**4b)** Explain what makes reinforcement learning different from supervised learning tasks such as regression or classification. 


Answer:

a) Exploration is important in reinforcement learning because it allows the agent to discover new and better policies. Without exploration the agent only relies on its current knowledge and may miss out on better policies.

For example, if we think of an agent learning to solve a rubics cube. If the agent always chooses the action with the highest expected reward based on its current policy, it may repeatedly follow the same pattern to get to a solution. However, it is not certain this solution is the most optional. By using an exploration strategy the agent may occasionally find another solution, which could be faster/more efficient.

b) The main difference between reinforcement learning (RL) and supervised learning (SL) is that RL learns by doing actions in an environement, and receiving rewards/penalties for these actions. It does not know the answer from the begnning, but continously gets closer to a solution. SL learns from already labeled input/output pairs, thus it already has the answers and tries to create a model which creates the most accurate solution for the dataset. RL data involves exploration-exploitation tradeoffs and often deals with problems that have a long-term reward, while SL data is often samples from an underlying distribution.


## Question 5

**5a)** Give a summary of how a decision tree works and how it extends to random forests.

**5b)** State at least one advantage and one drawback with using random forests over decision trees.

Answer:

**a)** A decision tree is a type of machine learning algorithm that can help predict outcomes based on input data. It does this by breaking down the data into features, and creating a set of rules to to make a prediction based on the input features. When these features get more specific the model gets more accurate and is able to make higher accuracy predictions based on the features. However this is prone to overfitting because the decision points or "questions" that gets formed by the model is very reliant on the dataset given and data from outside the dataset may not fit with the features in the decision tree. 

Random forests is an extension of decision trees that can help improve accuracy by using multiple decision trees, each trained on a random subset of the data and features. This can help to reduce overfitting and improve the robustness of the model.

Decision trees and random forests are both models of supervised learning.

**b)** One advantage of random forests over decision trees is that they are less prone to overfitting. Decision trees can easily overfit to the training data. Random forests overcome this issue by having multiple decision trees and combining their predictions. This reduces the variance of the model and often results in better performance on new data.

On the other hand, one potential drawback of using random forests is that they can be computationally expensive and slow to train. This is because random forests must train multiple decision trees and perform additional operations to combine their predictions.


# References
Primer/text based on the following references:
* http://www.cse.chalmers.se/~chrdimi/downloads/book.pdf
* https://github.com/olethrosdc/ml-society-science/blob/master/notes.pdf

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5c934c62-117f-45fe-8fd9-3a69f164f4d1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>